In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications import MobileNetV3Small

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = MobileNetV3Small(
    include_top=False,
    classes=len(train.class_names),
    classifier_activation=None,
    input_shape=inputShape
)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

dropoutCriticalPoints = (0,1)
adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3) 

n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    dropoutRate = basicUtil.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = transferUtil.createTransferableModel(base_model, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/MobileNetTrials/mobilenet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if (devScore > 0.91) and (devScore > bestDevScore):
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_mobilenet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'dropoutRate' : dropoutRate,
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_mobilenet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
    
        possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
        dropoutCriticalPoints = basicUtil.calculateCriticalPoints(top5['dropoutRate'])
        adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        dropoutRates = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_mobilenet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.
4334752/4334752 [==============================] - 63s 14us/step


2024-08-11 22:42:57.132854: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-08-11 22:42:57.133027: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-08-11 22:42:57.969142: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 9s 105ms/step - loss: 3.0957 - f1_score: 0.8209
devScore


2024-08-11 22:43:37.315105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-08-11 22:43:37.315246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 1s 79ms/step - loss: 3.0875 - f1_score: 0.8207
concluding trial  0
trainScore
75/75 [==============================] - 8s 102ms/step - loss: 0.6927 - f1_score: 0.9704
devScore
10/10 [==============================] - 1s 75ms/step - loss: 0.6917 - f1_score: 0.9704


2024-08-11 22:45:02.142540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]
2024-08-11 22:45:02.142647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 115ms/step - loss: 0.6938 - f1_score: 0.9704
concluding trial  1
trainScore
75/75 [==============================] - 9s 106ms/step - loss: 1.1310 - f1_score: 0.6941
devScore
10/10 [==============================] - 1s 73ms/step - loss: 1.2058 - f1_score: 0.6433
concluding trial  2
trainScore
75/75 [==============================] - 8s 102ms/step - loss: 0.1718 - f1_score: 0.9251
devScore
10/10 [==============================] - 1s 76ms/step - loss: 0.2179 - f1_score: 0.8959
concluding trial  3
trainScore
75/75 [==============================] - 9s 106ms/step - loss: 0.1070 - f1_score: 0.9715
devScore
10/10 [==============================] - 1s 67ms/step - loss: 0.1458 - f1_score: 0.9516
concluding trial  4
trainScore
75/75 [==============================] - 9s 105ms/step - loss: 3.2522 - f1_score: 0.5938
devScore
10/10 [==============================] - 1s 64ms/step - loss: 3.2787 - f1_score: 0.5955
concluding trial  5
trainScore
75/75 [=====

,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
6,14,7,0.998333,0.002281,747.031645,0.971736,0.971735
4,11,8,0.443968,0.000901,0.013827,0.971455,0.951610
1,24,9,0.926293,0.005528,925.452440,0.970424,0.970424
8,9,7,0.675746,0.000486,13.945878,0.956879,0.949171
3,4,8,0.381468,0.000580,0.045751,0.925071,0.895875
0,26,4,0.889671,0.002997,41.571168,0.820925,0.820674
2,22,1,0.388623,0.002089,1.094543,0.694101,0.643311
7,13,1,0.632930,0.000124,45.854116,0.669240,0.665691
5,27,1,0.335109,0.005897,3.285361,0.593811,0.595547


Final Model


,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,14,7,0.998333,0.002281,747.031645,7.202995,0.971736,0.971735,0.971735
